In [1]:
import os
from time import sleep
import pandas as pd
import csv
from sqlalchemy import create_engine
import sqlite3


In [3]:
conn = sqlite3.connect('mailchimp.db')
cursor = conn.cursor()

In [4]:
sql = """
SELECT DISTINCT S.CAMPAIGN, 
    S.DATE, 
    COUNT(S.EMAIL)  AS SENT_COUNT,
    COUNT(U.EMAIL)  AS UNOPEN_COUNT,
    COUNT(O.EMAIL)  AS OPEN_COUNT,
    COUNT(C.EMAIL)  AS CLICK_COUNT
FROM SENT S
LEFT JOIN OPEN O
    ON S.EMAIL = O.EMAIL AND S.CAMPAIGN = O.CAMPAIGN
LEFT JOIN CLICKED C
    ON S.EMAIL = C.EMAIL AND S.CAMPAIGN = C.CAMPAIGN
LEFT JOIN UNOPEN U
    ON S.EMAIL = U.EMAIL AND S.CAMPAIGN = U.CAMPAIGN
GROUP BY S.CAMPAIGN
ORDER BY S.DATE DESC
"""

In [5]:
query = pd.read_sql_query(sql, conn)
query

,CAMPAIGN,DATE,SENT_COUNT,UNOPEN_COUNT,OPEN_COUNT,CLICK_COUNT
0,006 - 03- Pro-Consult Legal - Your Contacts: M...,2022-01-13 18:00:00.000000,564,538,26,0
1,006 - 02- Pro-Consult Legal - Your Contacts: M...,2022-01-10 12:30:00.000000,668,565,103,10
2,005 - 02 - Pro-Consult - Real Estate Campaign ...,2022-01-05 18:00:00.000000,673,443,230,14
3,006 - Pro-Consult Legal - Your Contacts: More ...,2022-01-02 17:45:00.000000,677,510,167,15
4,005 - Pro-Consult - Real Estate Campaign,2021-12-29 14:40:00.000000,206,108,98,1
5,004 - Pro-Consult - Real Estate Campaign,2021-12-27 18:49:00.000000,474,189,285,16


In [6]:
#Data processing
query['WEEKDAY'] = pd.to_datetime(query['DATE']).dt.day_name()
query['HOUR'] = pd.to_datetime(query['DATE']).dt.hour
query['OPEN_RATE'] = (query['OPEN_COUNT'] / query['SENT_COUNT'])*100
query['CAMPAIGN_CODE'] = query['CAMPAIGN'].str[:3]
query


,CAMPAIGN,DATE,SENT_COUNT,UNOPEN_COUNT,OPEN_COUNT,CLICK_COUNT,WEEKDAY,HOUR,OPEN_RATE,CAMPAIGN_CODE
0,006 - 03- Pro-Consult Legal - Your Contacts: M...,2022-01-13 18:00:00.000000,564,538,26,0,Thursday,18,4.609929,006
1,006 - 02- Pro-Consult Legal - Your Contacts: M...,2022-01-10 12:30:00.000000,668,565,103,10,Monday,12,15.419162,006
2,005 - 02 - Pro-Consult - Real Estate Campaign ...,2022-01-05 18:00:00.000000,673,443,230,14,Wednesday,18,34.175334,005
3,006 - Pro-Consult Legal - Your Contacts: More ...,2022-01-02 17:45:00.000000,677,510,167,15,Sunday,17,24.667651,006
4,005 - Pro-Consult - Real Estate Campaign,2021-12-29 14:40:00.000000,206,108,98,1,Wednesday,14,47.572816,005
5,004 - Pro-Consult - Real Estate Campaign,2021-12-27 18:49:00.000000,474,189,285,16,Monday,18,60.126582,004


In [7]:
#OPEN_RATE BY DAY OF WEEK
query.groupby(['WEEKDAY']).mean()['OPEN_RATE'].sort_values(ascending=False)

WEEKDAY
Wednesday    40.874075
Monday       37.772872
Sunday       24.667651
Thursday      4.609929
Name: OPEN_RATE, dtype: float64

In [8]:
#OPEN_RATE BY HOUR OF DAY
query.groupby(['WEEKDAY','HOUR']).mean()['OPEN_RATE'].sort_values(ascending=False)

WEEKDAY    HOUR
Monday     18      60.126582
Wednesday  14      47.572816
           18      34.175334
Sunday     17      24.667651
Monday     12      15.419162
Thursday   18       4.609929
Name: OPEN_RATE, dtype: float64

In [9]:
#OPEN_RATE BY CAMPAIGN CODE
query.groupby(['CAMPAIGN_CODE']).mean()['OPEN_RATE']

CAMPAIGN_CODE
004    60.126582
005    40.874075
006    14.898914
Name: OPEN_RATE, dtype: float64

In [140]:
#OPEN RATE BY TIME AND CAMPAING
query.groupby(['WEEKDAY','HOUR','CAMPAIGN_CODE']).mean()['OPEN_RATE'].sort_values(ascending=False)

WEEKDAY    HOUR  CAMPAIGN_CODE
Monday     18    004              60.126582
Wednesday  14    005              47.572816
           18    005              34.175334
Sunday     17    006              24.667651
Monday     12    006              15.419162
Thursday   18    006               4.609929
Name: OPEN_RATE, dtype: float64